In [17]:
import os
import librosa
import soundfile as sf
import speech_recognition as sr

In [18]:
def get_text_from_audio(file_path):
    recognizer = sr.Recognizer()

    with sr.AudioFile(file_path) as source:
        audio_data = recognizer.record(source)
        text = recognizer.recognize_google(audio_data, language="ru-RU")

    return text

In [22]:
def split_audio_by_silence(input_folder, output_folder, max_files=10):
    for root, dirs, files in os.walk(input_folder):
        for file in files:
            if file.endswith(".wav"):
                file_path = os.path.join(root, file)

                # Загрузка аудиофайла
                audio, sr = librosa.load(file_path, sr=16000)

                # Разделение на части по местам тишины
                intervals = librosa.effects.split(audio, top_db=20)

                # Извлечение текста из аудио
                full_text = get_text_from_audio(file_path)

                # Создание папки для результатов
                for word in full_text.split():
                    result_folder = os.path.join(output_folder, word)
                    os.makedirs(result_folder, exist_ok=True)

                    # Ограничение до max_files файлов, если указано
                    for i, (start, end) in enumerate(intervals[:max_files]):
                        audio_part = audio[start:end]
                        txt_file = os.path.join(result_folder, f"{word}.txt")
                        wav_file = os.path.join(result_folder, f"{word}.wav")

                        # Сохранение текста в файл
                        with open(txt_file, 'w') as txt:
                            txt.write(full_text)

                        # Сохранение аудиофайла
                        sf.write(wav_file, audio_part, sr)

In [23]:
# Замените пути к папкам на ваши
input_folder = "/home/redalexdad/recognition_speech/new_buriy_audiobooks_2_val"
output_folder = "/home/redalexdad/recognition_speech/voice_split"

In [24]:
split_audio_by_silence(input_folder, output_folder, max_files=10)

In [25]:
from IPython import display
import matplotlib.pyplot as plt

In [26]:
data_dir = '/home/redalexdad/recognition_speech/voice_split'

# label_names = []
file_counter = 0  # Переменная для отслеживания количества обработанных файлов

# Рекурсивно обходим все файлы и подкаталоги в корневом каталоге
for root, dirs, files in os.walk(data_dir):
    for file in files:
        # Проверяем, что файл - звуковой файл
        if file.endswith(".wav"):
            # Увеличиваем счетчик обработанных файлов
            file_counter += 1

            # Проверяем, не превысило ли количество файлов 10
            if file_counter > 10:
                break

            # Остальной код вашего скрипта остается без изменений
            audio_file_path = os.path.join(root, file)
            label = os.path.basename(root)
            text_file_path = os.path.join(root, f"{os.path.splitext(file)[0]}.txt")

            if not os.path.exists(text_file_path):
                print(f"No text file found for {audio_file_path}")
                continue

            # Далее ваш код для чтения текста и вывода информации о звуке
            print(f"Label {label}")
            waveform, sample_rate = librosa.load(audio_file_path, sr=None)

            print("Audio shape:", waveform.shape)
            print('Waveform shape:', waveform.shape)
            # display(waveform)
            print('Audio playback')
            display.display(display.Audio(waveform, rate=sample_rate))

            plt.figure(figsize=(16, 3))
            plt.plot(waveform)
            plt.title(label)
            plt.yticks(np.arange(-1.2, 1.2, 0.2))
            plt.ylim([-1.1, 1.1])
            # Отображаем график после обработки
            plt.show()